# Vinyl Grading Project
This project aims to construct an open source package to classify vinyl according to the [Goldmine Grading](https://www.goldminemag.com/collector-resources/record-grading-101).

First we need to create a Dataset. We will scrap some pages in search of labeled audio.

## Scrapin WatchCount

WatchCount is an archive of ebay sold items. I've selected one user that uses a template for the description and that includes always an audio. This will help us to create semi-automatically the dataset.

In [1]:
# install required packages if needed
!pip install pandas selenium requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 6.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 6.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 6.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 6.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 6.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 4.8 

In [2]:
# Imports
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [4]:
# Let's open a session
driver = webdriver.Firefox()
url = "http://www.watchcount.com/completed.php?bfw=1&bslr=wills-rare-records"
driver.get(url)



Let's create a dataframe for the url to the actual item on ebay and the title of the item sold.

In [4]:
df_links = pd.DataFrame(columns=["Link","Title"])

In [5]:
df_links

,Link,Title


Now is time to scrape all the table of sold items from this seller.

In [13]:
# This is the table's xpath
table = driver.find_element(By.XPATH, "/html/body/div/table/tbody/tr[3]/td/table/tbody/tr/td[2]/div/table[2]/tbody/tr[2]/td/div/table[1]")

# Find all rows within the table
rows = table.find_elements(By.TAG_NAME, "tr")

# Loop through each row
for row in rows:
    # Find all link elements within the row
    links = row.find_elements(By.TAG_NAME, "a")
    # Loop through each link element
    if len(links)==11:

    # Extract the URL from the href attribute
        url = links[1].get_attribute("href")
        text = links[1].get_attribute("text")
        
        df_links.loc[len(df_links)] = [url, text]


Let's have a look on the dataframe of the first scraped page.

In [14]:
df_links

,Link,Title
0,http://www.watchcount.com/go/?item=11623035299...,Ahmed al-Jaberi - Rare SUDAN Arabic Afro 45 / ...
1,http://www.watchcount.com/go/?item=11623035299...,Ahmed al-Jaberi - Rare SUDAN Arabic Afro 45 / ...
2,http://www.watchcount.com/go/?item=11623035039...,Mohamed Mirghani - Rare SUDAN Arabic Afro 45 /...
3,http://www.watchcount.com/go/?item=11623034962...,Tayeb Abdullah ? - Rare SUDAN Arabic Afro 45 /...
4,http://www.watchcount.com/go/?item=11623034725...,Ibrahim Awad - Ya Zaman - Rare SUDAN Arabic Af...
...,...,...
96,http://www.watchcount.com/go/?item=11620727474...,Asnaketch Worku – Krar Songs - ETHIOPIA Rare F...
97,http://www.watchcount.com/go/?item=11620668755...,Tekle Tesfazghi ‎– Abadit - ERITREA (Ethiopia)...
98,http://www.watchcount.com/go/?item=11620727128...,Ashenafi Kebede & The Hungarian State String O...
99,http://www.watchcount.com/go/?item=11620668101...,Black Magic Band - ZAMBIA Invisible Super Rare...


Look's good. Let's scrape the rest of 8 pages.

In [17]:
for i in range(1,9):
    driver = webdriver.Firefox()
    url = "http://www.watchcount.com/completed.php?bslr=wills-rare-records&csbin=all&cssrt=ts&bfw=1&bpg={}#serp".format(i)
    driver.get(url)
    table = driver.find_element(By.XPATH, "/html/body/div/table/tbody/tr[3]/td/table/tbody/tr/td[2]/div/table[2]/tbody/tr[2]/td/div/table[1]")

    # Find all rows within the table
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Loop through each row
    print(i)
    for row in rows:
        try:
            # Find all link elements within the row
            links = row.find_elements(By.TAG_NAME, "a")
            
            # Loop through each link element
            if len(links)==11:

                # Extract the URL from the href attribute
                url = links[1].get_attribute("href")
                text = links[1].get_attribute("text")
                df_links.loc[len(df_links)] = [url, text]
        except:
            print(row)
driver.quit()

4


Let's store the output.

In [19]:
df_links.to_csv("./output/wills.csv", index = False)